In [1]:
import os
import pandas as pd
import numpy as np
import awkward as ak
import uproot_methods
print(ak.__version__)

0.12.0


In [2]:
##both should be numpy array
ls = np.array([1,2,1])
mask = np.array([2,1])
a = ak.JaggedArray.fromcounts(mask, ls)

In [24]:
##and Professor suggests that we could use mass, classifacation for later application
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    n_particles_ls = []
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    _label1 = []
    _label2 = []
    n = 0
    for line in lines:
        if line.startswith('E'):
            if not n == 0:
                n_particles_ls.append(n)
                n = 0
            exp_inf = line.split()
            _label1.append(int(exp_inf[1]))
            _label2.append(1-int(exp_inf[1]))
        else:
            par = line.split()
            ##particle +1
            n = n + 1
            px_ls.append(float(par[1]))
            py_ls.append(float(par[2]))
            pz_ls.append(float(par[3]))
            energy_ls.append(float(par[4]))
            mass_ls.append(float(par[5]))
    if not n == 0:
        n_particles_ls.append(n)
    px_arr = np.array(px_ls)
#     print(px_arr)
    py_arr = np.array(py_ls)
    pz_arr = np.array(pz_ls)
    energy_arr = np.array(energy_ls)
    mass_arr = np.array(mass_ls)
    n_particles = np.array(n_particles_ls)
#     print(n_particles)
    px = ak.JaggedArray.fromcounts(n_particles, px_arr)
#     print(px)
    py = ak.JaggedArray.fromcounts(n_particles, py_arr)
    pz = ak.JaggedArray.fromcounts(n_particles, pz_arr)
    energy = ak.JaggedArray.fromcounts(n_particles, energy_arr)
    mass = ak.JaggedArray.fromcounts(n_particles, mass_arr)
    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    ##Create an Order Dic
    from collections import OrderedDict
    v = OrderedDict()
    v['part_px'] = px
#     print(px)
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy
    v['part_mass'] = mass
#     ls1 = [1,2,3,4]
#     ls2 = [5,6,7,8]
    v['label'] = np.stack((_label1, _label2), axis=-1)
#     print(v['label'])
    return v

In [25]:
x = SetAKArr("train.txt")

[19 19 11 11  8 12  3 19 12 10 18 13  9 14  4  7 22  9 20  6 14  9  7 12
  7 12 10 13  6  9 18 14 13 11 17  8 10  7  6 11 14  6 11 11  9 13 18 16
  7  8  7 13 10 19  6 14 13  7  9 14  9 19 13 10 11 11 10  9 10  9 11  8
 21 11 14  8 10 13 14 15 11 10  8 12 11 18 14 15 10  7 12 18 12  9  6 15
 15 14  6 11 14 15 10 11  7 11  8 11 11  7 19 16 13  9 19 13  9 11 11 10
  8 21  9  8  6 17 14 23 20  9 14 15  9 12 11 10  9 10  6  7 12 11 10  4
 10  7 15  8 13  7 10  6 13 11 11 12 17  8 15 18 15 14 11 10 15 10 12 12
 11 16 12  8  9 10  7  7  8 13 11  8  9  9 18  8  9 12 12 13 12 16  9 18
 11  9 18 12  8 11 21 13  8 11 12 13 15 28  8  6  8 16 18 16 13 13  9  9
 11 16  6 18 10 10  9 14  8 12 22  8 12  8 14 11 10 15 10  8  7 11  9  9
 15 17 13 10 16 10 11 11  9 12 13 17  8  9 13 15 10 17 17 10 10 10 12 10
 10 15  4  9 15 16 16 10 17 18 10  9 13  7 11 11 17 13 10 16 21 15 12 20
  8 10  7 23 11  9  9  9  9 16 10 16 15 12 13  6  8  9 13  9 19  7 15 15
 16 12  8 10 13 13 13 15 11 12  7 20  8 11 10 11 15

In [2]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
print(myarray[1:])
# returns <JaggedArray [[] [4.4 5.5]] at 7f02018afc18>
print(myarray[100:])
# returns <JaggedArray [] at 7f020c214438>

TypeError: a bytes-like object is required, not 'list'

In [4]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
indexes = ak.JaggedArray.fromiter([[1, 2, 2, 0], [], [1]])
myarray[indexes]
# returns <JaggedArray [[3.3 3.3 1.1] [] [5.5]] at 7f02018afa58>

TypeError: a bytes-like object is required, not 'list'

In [14]:
myarray = ak.JaggedArray.fromcounts([3, 0, 2], ak.Table(
              x=[1, 2, 3, 4, 5],
              y=[1.1, 2.2, 3.3, 4.4, 5.5],
              z=[True, False, True, False, False]))
print(myarray["x"])
# returns <JaggedArray [[1 2 3] [] [4 5]] at 7f020e8122b0>
print(myarray[["x", "y"]])
# returns <JaggedArray [[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]] at 7f02018af860>
print(myarray[["x", "y"]].columns)
# returns ['x', 'y']
print(myarray.count())
print(myarray)
myarray["x"] = [1, 2, 3, 4, 5]
print(myarray["x"])

[[1 2 3] [] [4 5]]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
['x', 'y']
[<Row 0> <Row 1> <Row 2>]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
[[1 1 1] [] [3 3]]
